# DB2-Salesforce connector: Basic user information updates

%load_ext autoreload
%autoreload 2

In [1]:
# Parameters
hours_range = 24*365*1 # number of hours to look back

# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login credentials
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"mathisfun992728tXYwCb5yMm6XtohbTRw8ZdBbi",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",


In [2]:
import sys
#sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Obtain contacts from DB2 
that have last visit date within range of interest specified by day_range

In [3]:
# Depending on the task, use different query and cutoff

# Hourly update for new registrations
date_cutoff = (datetime.datetime.today().date() - datetime.timedelta(hours=hours_range))\
                    .strftime('%Y-%m-%d')
sql_query = "select id, name, username, block, email, sendEmail, registerDate, lastvisitDate \
                    from jos_users where registerDate >= '%s'" % date_cutoff

# display
print(sql_query)

select id, name, username, block, email, sendEmail, registerDate, lastvisitDate                     from jos_users where registerDate >= '2019-08-25'


In [11]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine)

# get user profile details
profile_df = pd.read_sql_query("select * from jos_user_profiles where profile_key \
in ('orgtype', 'organization', 'orcid')", engine)



In [18]:
# display
display(df.head(1))

display(profile_df.head(2))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate
0,258846,Michael Thomas Thoenen,mthoenen,0,mthoenen@purdue.edu,0,2019-08-25 01:10:27,2019-11-12 20:14:51


,id,user_id,profile_key,profile_value,ordering,access
0,1,15623,orgtype,university,2,5
1,2,10060,orgtype,university,2,5


In [7]:
df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orgtype'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orgtype'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'organization'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'organization'})
display(df.head(1))

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orcid'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orcid'})

# display
display(df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization
0,258426,Sungmin Na,nasum621,0,nasum621@gmail.com,-1,2019-08-21 00:06:12,2019-08-26 09:15:11,universitygraduate,NaN


,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid
0,258426,Sungmin Na,nasum621,0,nasum621@gmail.com,-1,2019-08-21 00:06:12,2019-08-26 09:15:11,universitygraduate,NaN,NaN


In [8]:
# Obtain company domain information from DB2
domain_df = pd.read_sql_query("select name as domain_name, domain, industry, `size range` as size, country \
from wang159_myrmekes.companies_email_domain", engine)

In [9]:
# make sure domain is unique and drop NaN
domain_df = domain_df.drop_duplicates(subset='domain', keep='first').dropna()

In [10]:
# get email domain of nanoHUB users
def get_domain(this_email):
    seg_list = this_email.split('@')
    
    if len(seg_list) == 2:
        return seg_list[1].lower()
    else:
        return None
    
df['email_host'] = df['email'].apply(get_domain)

In [11]:
# save a constuct for debugging
from copy import deepcopy
temp = deepcopy(df)

In [12]:
# get domain by nanoHUB organization field
df = deepcopy(temp)
display(df.head(2))
df['organization'] = df['organization'].str.lower()
df = df.fillna('   ')

df.loc[(df['organization']=='nan'),'organization'] = None
df.loc[(df['organization']=='NaN'),'organization'] = None

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host
0,258426,Sungmin Na,nasum621,0,nasum621@gmail.com,-1,2019-08-21 00:06:12,2019-08-26 09:15:11,universitygraduate,NaN,NaN,gmail.com
1,258427,BeJimmieClupt BeJimmieClupt BeJimmieClupt,bejimmieclupt,0,opetr5401@gmail.com,1,2019-08-21 00:08:02,2019-08-30 22:51:33,universityundergraduate,NaN,NaN,gmail.com


display(df.tail())

In [13]:
import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
s_words = stopwords.words('english')

In [14]:
import re

def clean_domain(this_domain_name):
    
    if not this_domain_name:
        return None
    
    # Replace all non-alphanumeric characters with space
    this_domain_name = re.sub("[^0-9a-zA-Z]+", " ", this_domain_name.lower())
    name_list = this_domain_name.split(' ')
        
    # remove all stop words
    name_list = ['' if x in s_words else x for x in name_list]
    
    return set(filter(None, name_list))

In [15]:
display(domain_df.head(2))

,domain_name,domain,industry,size,country
0,strategic interns,strategicinterns.com,education management,1-10,india
1,bosque y comunidad,bosqueycomunidad.org,non-profit organization management,1-10,spain


In [16]:
# clean domain name
domain_df['domain_cleaned_set'] = domain_df.domain_name.apply(clean_domain)
display(domain_df.head(1))
# clean nanohub org name
df['org_cleaned_set'] = df.organization.apply(clean_domain)
display(df.head(1))

,domain_name,domain,industry,size,country,domain_cleaned_set
0,strategic interns,strategicinterns.com,education management,1-10,india,"{interns, strategic}"


,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set
0,258426,Sungmin Na,nasum621,0,nasum621@gmail.com,-1,2019-08-21 00:06:12,2019-08-26 09:15:11,universitygraduate,,,gmail.com,{}


In [17]:
# hash cleaned domain name
domain_df['domain_cleaned_hash'] = domain_df['domain_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

# hash cleaned clean nanohub org name
df['org_cleaned_hash'] = df['org_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

In [18]:
display(domain_df.head(1))
display(df.head(1))

,domain_name,domain,industry,size,country,domain_cleaned_set,domain_cleaned_hash
0,strategic interns,strategicinterns.com,education management,1-10,india,"{interns, strategic}",interns-strategic


,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash
0,258426,Sungmin Na,nasum621,0,nasum621@gmail.com,-1,2019-08-21 00:06:12,2019-08-26 09:15:11,universitygraduate,,,gmail.com,{},None


## Collaborative filtering for commerical email hosts

In [19]:
def predict_if_commerical(this_email_host_df):
    # predict if this email host is commerical
    top_cnt = this_email_host_df['org_cleaned_hash'].value_counts() #returns uniques in descending order
    
    if top_cnt.shape[0] > 0:
        if (top_cnt.iloc[0]/top_cnt.sum()) < 0.2:
            return True
        else:
            return False
    
    else:
        return False
    
    
is_email_commercial = df[['email_host', 'org_cleaned_hash']].groupby('email_host').apply(predict_if_commerical)

In [20]:
temp2 = deepcopy(df)

In [21]:
df = deepcopy(temp2)

In [22]:
# mark commerical email addresses
df = pd.merge(df, is_email_commercial.reset_index(name='is_email_commercial'),\
         how='left', left_on='email_host', right_on='email_host' ,)

In [23]:
display(df.tail(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial
28352,297333,Li Tang,tangli666,0,tangli666@gmail.com,0,2020-08-20 15:30:53,2020-08-20 15:33:13,industry,,,gmail.com,{},None,True


In [24]:
# obtain domain estimation from email addresses
df = pd.merge(df, domain_df[~domain_df.domain.isin(is_email_commercial[is_email_commercial].index)]['domain'],\
         how='left', left_on='email_host', right_on='domain').rename(columns={'domain':'domain_by_email'})

# display
display(df.head(2).T)

,0,1
id,258426,258427
name,Sungmin Na,BeJimmieClupt BeJimmieClupt BeJimmieClupt
username,nasum621,bejimmieclupt
block,0,0
email,nasum621@gmail.com,opetr5401@gmail.com
sendEmail,-1,1
registerDate,2019-08-21 00:06:12,2019-08-21 00:08:02
lastvisitDate,2019-08-26 09:15:11,2019-08-30 22:51:33
orgtype,universitygraduate,universityundergraduate
organization,,


## Fuzzy derive organization from nanoHUB profiles

In [25]:
# attempt direct join by hash
domain_subset_df = domain_df[domain_df.domain.isin(df.email_host.unique())][['domain', 'domain_cleaned_hash']]
# only take values that exist
domain_subset_df = domain_subset_df[~domain_subset_df.domain.isna()&~domain_subset_df.domain_cleaned_hash.isna()]

derived_df = pd.merge(df, domain_subset_df.rename(columns={'domain':'domain_by_profile'}),\
         how='left', left_on='org_cleaned_hash', right_on='domain_cleaned_hash')\
        .drop('domain_cleaned_hash', axis=1)

# display
derived_df.sample(5).T

,21659,12129,12084,2607,19204
id,289104,277090,277037,262312,286112
name,Christian Massao Konishi,Edrian Tirene,Nils Materna,Currin Castro,unsur totovip77
username,c214570,edrian.tirene,slinanretam1,bettie3738albertha,unsurtoto99
block,0,0,0,0,1
email,c214570@dac.unicamp.br,edrian.tirene@ufl.edu,slinanretam@gmail.com,lisagaughfjcgns@yahoo.com,unsurtotovip77@gmail.com
sendEmail,0,0,-1,1,-1
registerDate,2020-05-25 18:17:40,2020-01-30 17:57:12,2020-01-30 08:19:58,2019-09-08 09:42:05,2020-04-28 12:56:38
lastvisitDate,2020-05-26 21:35:35,2020-03-27 20:52:26,2020-01-30 08:19:58,2020-08-18 16:56:37,2020-04-28 12:56:38
orgtype,universityundergraduate,universityundergraduate,universityundergraduate,universitygraduate,industry
organization,,university of florida,,,


In [26]:
# attempt to reduce domain_df size to speedup Jaccard calculation
from itertools import chain

all_word_set = set(chain.from_iterable(derived_df[(derived_df['domain_by_profile'].isna() & (~derived_df['organization'].isna()))]\
                    .org_cleaned_set.values))

# select the domain_df entry with words from all_word_set
domain_subset_df = domain_df[domain_df.domain_cleaned_set.notnull()]
domain_subset_df = domain_subset_df[domain_subset_df.domain_cleaned_set\
                             .apply(lambda x: True if len(x.intersection(all_word_set)) > 0 else False)]

In [27]:
display(domain_subset_df.head(2))

,domain_name,domain,industry,size,country,domain_cleaned_set,domain_cleaned_hash
4,"cardinal strategies, llc",cardinal-strategies.com,civil engineering,11-50,united states,"{llc, strategies, cardinal}",cardinal-llc-strategies
16,grail research,grailresearch.com,research,201-500,united states,"{research, grail}",grail-research


In [28]:
def get_org(this_df):
    # look at the cohort with this label, do they have a commonly agreed domain_by_email? If so, use it
    this_dbe_count = this_df.domain_by_email.value_counts()

    if this_dbe_count.shape[0] < 1:
        return None

    this_top_dbe_perc = this_dbe_count.iloc[0]/this_dbe_count.sum()

    # if top choice occupies majority of cases, then this cohort can be implied to this domain_by_email address
    if this_top_dbe_perc > 0.5:
        return this_dbe_count.index[0] 


derived_hash = derived_df.groupby('org_cleaned_hash').apply(get_org)

In [29]:
derived_df = pd.merge(derived_df, derived_hash.reset_index(name='domain_by_profile_infer')\
                      , how='left', left_on='org_cleaned_hash', right_on='org_cleaned_hash')

In [30]:
# calculate a composite domain based on domain_by_email, domain_by_profile, and domain_by_profile_infer
def get_composite_domain(this_df):

    # if domain found via profile with direct match, use it
    if this_df.domain_by_profile != '':
        return this_df.domain_by_profile
    
    # if domain found via email, use it
    if this_df.domain_by_email != '':
        return this_df.domain_by_email    
    
    # if domain found via profile but not with a direct match, use it
    if this_df.domain_by_profile_infer != '':
        return this_df.domain_by_profile_infer
    
    # else, no organization can be concluded, return none
    return None

    
derived_df['domain_final'] = derived_df[['domain_by_email', 'domain_by_profile', 'domain_by_profile_infer']].fillna('')\
                                                        .apply(get_composite_domain, axis=1)

In [31]:
display(derived_df.head(2))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,258426,Sungmin Na,nasum621,0,nasum621@gmail.com,-1,2019-08-21 00:06:12,2019-08-26 09:15:11,universitygraduate,,,gmail.com,{},None,True,NaN,NaN,NaN,None
1,258427,BeJimmieClupt BeJimmieClupt BeJimmieClupt,bejimmieclupt,0,opetr5401@gmail.com,1,2019-08-21 00:08:02,2019-08-30 22:51:33,universityundergraduate,,,gmail.com,{},None,True,NaN,NaN,NaN,None


## Update Salesforce institution 

In [32]:
# get a unique list of organizations that appears in contact
#org_df = derived_df[['domain_name', 'domain', 'industry', 'size', 'country']].drop_duplicates().dropna()
org_df = pd.merge(derived_df[['domain_final']].drop_duplicates().dropna(), domain_df, how='inner', left_on='domain_final', right_on='domain') 
org_df = org_df[['domain_name', 'domain', 'industry', 'size', 'country']]

# display
display(org_df.head(2))

,domain_name,domain,industry,size,country
0,purdue university,purdue.edu,higher education,10001+,united states
1,vanderbilt university,vanderbilt.edu,higher education,10001+,united states


In [33]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name'] = org_df['domain_name']

df_sf['Size_Range__c'] = org_df['size'].fillna('Unknown')
df_sf['Country__c'] = org_df['country']
df_sf['Domain__c'] = org_df['domain']
df_sf['Industry__c'] = org_df['industry']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,purdue university,vanderbilt university
Size_Range__c,10001+,10001+
Country__c,united states,united states
Domain__c,purdue.edu,vanderbilt.edu
Industry__c,higher education,higher education


In [34]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = 'organization__c'
db_s.external_id = 'Domain__c'

Obtained Salesforce access token ...... True


In [35]:
display(df_sf.head(2))
display(df_sf.tail(2))


,Name,Size_Range__c,Country__c,Domain__c,Industry__c
0,purdue university,10001+,united states,purdue.edu,higher education
1,vanderbilt university,10001+,united states,vanderbilt.edu,higher education


,Name,Size_Range__c,Country__c,Domain__c,Industry__c
858,icfo,501-1000,spain,icfo.eu,research
859,maltepe university,1001-5000,turkey,maltepe.edu.tr,higher education


In [36]:
## correcting name overflows
names = df_sf['Name'].str.len()

sf_char_limit = 80
name_indexes = names.index[names >= sf_char_limit].tolist()
for index in name_indexes:
    df_sf['Name'][index] =  df_sf['Name'][index][0:sf_char_limit]

In [37]:
display(df_sf.head(2))

,Name,Size_Range__c,Country__c,Domain__c,Industry__c
0,purdue university,10001+,united states,purdue.edu,higher education
1,vanderbilt university,10001+,united states,vanderbilt.edu,higher education


In [38]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000PRm6uAAD
hello
[Success] CSV upload successful. Job ID = 7505w00000PRm6uAAD
[Success] Closing job successful. Job ID = 7505w00000PRm6uAAD


In [41]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000PRm6uAAD',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-20T15:54:04.000+0000',
 'systemModstamp': '2020-08-20T15:54:11.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Domain__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 860,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 2632,
 'apiActiveProcessingTime': 2212,
 'apexProcessingTime': 0}

In [42]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

'"sf__Id","sf__Error",Country__c,Domain__c,Industry__c,Name,Size_Range__c\n'


## Obtain newly updated organization IDs from Salesforce

In [43]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [44]:
# get Salesforce ID for organizations
sf_org_ID_df = db_s.query_data('SELECT Id, Domain__c FROM organization__c')

# display
sf_org_ID_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000PRlqEAAT
{"id":"7505w00000PRlqEAAT","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-20T15:54:20.000+0000","systemModstamp":"2020-08-20T15:54:21.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000PRlqEAAT","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-20T15:54:20.000+0000","systemModstamp":"2020-08-20T15:54:22.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6259,"retries":0,"totalProcessingTime":580}
[Success] Bulk job completed successfully.


,Domain__c,Id
0,upenn.edu,a0r5w00000V42c0AAB
1,fer.hr,a0r5w00000V42c1AAB
2,wheatoncollege.edu,a0r5w00000V42c2AAB


In [45]:
# join salesforce ID back to contact DF
df = pd.merge(sf_org_ID_df, derived_df, how='right', left_on='Domain__c', right_on='domain_final')\
            .drop('Domain__c', axis=1)\
            .rename(columns={'Id':'Salesforce_org_ID'})

In [46]:
# display
display(df.head(2))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,a0r5w00000V42c0AAB,268121,"Thiruvengadam, Rangarajan Narasimhan",nitin17,0,nitin17@upenn.edu,0,2019-10-09 16:00:08,2019-10-09 16:00:08,universitygraduate,,,upenn.edu,{},None,False,upenn.edu,NaN,NaN,upenn.edu
1,a0r5w00000V42c0AAB,273295,"Cai, Yiyu",caiyiyu,0,caiyiyu@seas.upenn.edu,0,2019-12-08 23:30:54,2020-03-27 22:12:53,universitystaff,university of pennsylvania,,seas.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,None,upenn.edu


## Match data with Salesforce format

In [47]:
# split full name into first, middle, and last names
def split_full_name(this_name):
    this_name_list = list(filter(None, this_name.split(' ')))
    
    if len(this_name_list) == 1:
        # single word name
        return pd.Series([this_name_list[0],None, this_name_list[0]])
    
    elif len(this_name_list) > 1:
        # multi word name
        return pd.Series([this_name_list[0],' '.join(this_name_list[1:-1]),this_name_list[-1]])

In [48]:
## stop here
display(df.head(1))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,a0r5w00000V42c0AAB,268121,"Thiruvengadam, Rangarajan Narasimhan",nitin17,0,nitin17@upenn.edu,0,2019-10-09 16:00:08,2019-10-09 16:00:08,universitygraduate,,,upenn.edu,{},None,False,upenn.edu,NaN,NaN,upenn.edu


In [49]:
## if last visit date is empty - set it equal to the register date
lvd = df.lastvisitDate.index[df.lastvisitDate == '   '].tolist()

for i in lvd:
    df.iloc[i,8] = df.iloc[i,7]

display(df.head(2))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,a0r5w00000V42c0AAB,268121,"Thiruvengadam, Rangarajan Narasimhan",nitin17,0,nitin17@upenn.edu,0,2019-10-09 16:00:08,2019-10-09 16:00:08,universitygraduate,,,upenn.edu,{},None,False,upenn.edu,NaN,NaN,upenn.edu
1,a0r5w00000V42c0AAB,273295,"Cai, Yiyu",caiyiyu,0,caiyiyu@seas.upenn.edu,0,2019-12-08 23:30:54,2020-03-27 22:12:53,universitystaff,university of pennsylvania,,seas.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,None,upenn.edu


In [50]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf[['firstname', 'Middle_name__c', 'lastname']] = df['name'].apply(split_full_name)

df_sf['nanoHUB_user_ID__c']            = df['id']
df_sf['nanoHUB_username__c']           = df['username']
df_sf['Email']                         = df['email'].fillna('').apply(lambda x: '' if '@invalid' in x else x)

# for sendEmail: 0 = opt-out, 1 = receive email. For salesforce HasOptedOutOfEmail, it's exact opposite
df_sf['HasOptedOutOfEmail']            = df['sendEmail'].apply(lambda x: 0 if x==1 else 1)
df_sf['nanoHUB_account_BLOCKED__c']    = df['block'].fillna(0)

# solidify time-related columns from datetime to string
df_sf['nanoHUB_registration_date__c']  = df['registerDate'].dt.date.fillna('').astype('str')
df_sf['nanoHUB_last_active_date__c']   = df['lastvisitDate'].dt.date.fillna('').astype('str')

# Tableau detailed view
df_sf['Detailed_user_timeline_to_Tableau__c'] = df_sf['Email'].apply(lambda x: 'https://tableauqa.itap.purdue.edu/views/profile/ProfileTimeline?Id%20Email='+x+'\
&:iframeSizedToWindow=true&:embed=y&:showAppBanner=false\
&:display_count=no&:showVizHome=no#6' if x != '' else '')

# nanoHUB user profile
df_sf['nanoHUB_user_page__c'] = df_sf['nanoHUB_user_ID__c'].apply(lambda x: 'https://nanohub.org/members/%d'%x if x != '' else '')

df_sf['ORCID__c'] = df['orcid'].fillna('')
df_sf['Organization__c'] = df['organization'].fillna('')

# derived information
df_sf['Organization_email_derived__c'] = df['Salesforce_org_ID']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
firstname,"Thiruvengadam,","Cai,"
Middle_name__c,Rangarajan,
lastname,Narasimhan,Yiyu
nanoHUB_user_ID__c,268121,273295
nanoHUB_username__c,nitin17,caiyiyu
Email,nitin17@upenn.edu,caiyiyu@seas.upenn.edu
HasOptedOutOfEmail,1,1
nanoHUB_account_BLOCKED__c,0,0
nanoHUB_registration_date__c,2019-10-09,2019-12-08
nanoHUB_last_active_date__c,2019-10-09,2020-03-27


In [51]:
## filtering out accounts that have positive block attributes
spam_rows = df_sf[df_sf['nanoHUB_account_BLOCKED__c'].astype(str).str.contains("1")]
good_rows = df_sf[~df_sf['nanoHUB_account_BLOCKED__c'].astype(str).str.contains('1')]

## To Salesforce Sales Cloud CRM

In [52]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [53]:
display(spam_rows.head(2))
display(good_rows.tail(2))

,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Organization_email_derived__c
957,Mark,,Calaway,283456,markcalaway4,2r1st141@gmail.com,1,1,2020-04-07,2020-04-07,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/283456,,purdue university,a0r5w00000V42cCAAR
1266,John,,Neesham,295831,neeshamvm,johnneeshamvm@gmail.com,0,1,2020-08-03,2020-08-03,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/295831,,purdue university,a0r5w00000V42cCAAR


,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Organization_email_derived__c
28426,Bhuvaneswari,,Bunny,297332,bhuvaneswaribunny0813,bhuvaneswaribunny0813@gmail.com,1,0,2020-08-20,2020-08-20,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297332,,,NaN
28427,Li,,Tang,297333,tangli666,tangli666@gmail.com,1,0,2020-08-20,2020-08-20,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297333,,,NaN


In [54]:
# send data to Salesforce
db_s.send_data(good_rows)

[Success] Bulk job creation successful. Job ID = 7505w00000PRlNWAA1
hello
[Success] CSV upload successful. Job ID = 7505w00000PRlNWAA1
[Success] Closing job successful. Job ID = 7505w00000PRlNWAA1


In [74]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000PRlNWAA1',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-20T15:55:36.000+0000',
 'systemModstamp': '2020-08-20T15:57:52.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 27242,
 'numberRecordsFailed': 8,
 'retries': 0,
 'totalProcessingTime': 276168,
 'apiActiveProcessingTime': 245037,
 'apexProcessingTime': 2677130}

In [75]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",Detailed_user_timeline_to_Tableau__c,Email,HasOptedOutOfEmail,Middle_name__c,ORCID__c,Organization__c,Organization_email_derived__c,firstname,lastname,nanoHUB_account_BLOCKED__c,nanoHUB_last_active_date__c,nanoHUB_registration_date__c,nanoHUB_user_ID__c,nanoHUB_user_page__c,nanoHUB_username__c\n'
 '"","STRING_TOO_LONG:ORCID: data value too large: &lt;div itemscope '
 'itemtype=&quot;https://schema.org/Person&quot;&gt;&lt;a '
 'itemprop=&quot;sameAs&quot; '
 'content=&quot;https://orcid.org/0000-0002-9470-3078&quot; '
 'href=&quot;https://orcid.org/0000-0002-9470-3078&quot; '
 'target=&quot;orcid.widget&quot; rel=&quot;me noopener noreferrer&quot; '
 'style=&quot;vertical-align:top;&quot;&gt;&lt;img '
 'src=&quot;https://orcid.org/sites/default/files/images/orcid_16x16.png&quot; '
 'style=&quot;width:1em;margin-right:.5em;&quot; alt=&quot;ORCID iD '
 'icon&quot;&gt;https://orcid.org/0000-0002-9470-3078&lt;/a&gt;&lt;/div&gt; '
 '(max length=100):ORCID__c '
 '--","h

In [76]:
display(good_rows.tail(2))
print(good_rows.shape)

,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Organization_email_derived__c
28426,Bhuvaneswari,,Bunny,297332,bhuvaneswaribunny0813,bhuvaneswaribunny0813@gmail.com,1,0,2020-08-20,2020-08-20,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297332,,,NaN
28427,Li,,Tang,297333,tangli666,tangli666@gmail.com,1,0,2020-08-20,2020-08-20,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297333,,,NaN


(27242, 15)


## Verification
Checking to see if salesforce has updated contacts with the new users

request name and nanohub_user_id